# Triage rules baseline

Prototype notebook for rule-based clinical NLP triage.

- Loads `data/lexicon_redflags.csv`
- Loads `data/notes_synthetic.csv`
- Applies simple rule-based scoring
- Exports `outputs/predictions.csv`


In [ ]:
import pandas as pd

lexicon = pd.read_csv("../data/lexicon_redflags.csv")
notes = pd.read_csv("../data/notes_synthetic.csv")

terms = [str(t).lower() for t in lexicon["term"].tolist()]

def predict_label(text):
    text = str(text).lower()
    hits = sum(1 for t in terms if t and t in text)
    if hits >= 2:
        return "high"
    elif hits == 1:
        return "intermediate"
    else:
        return "low"

notes["predicted_label"] = notes["text"].apply(predict_label)

notes[["id", "text", "entity", "predicted_label"]].to_csv(
    "../outputs/predictions.csv", index=False
)

print("Saved outputs/predictions.csv")
